# 정형 데이터와 비정형 데이터
지금까지 우리가 다루어왔던 자료들이 전부 정형데이터였다.  
정형데이터란  길이, 높이, 무게 등을 데이터로 사용하는 것이다.  
뿐만 아니라 프로그래머가 다루는 대부분의 데이터가 정형 데이터이다.  
비정형 데이터란 액셀이나 데이터베이스로 표현하기 어려운 것들이다.  
예를 들어, 글, 사진, 음악 등이 있다.  
  
정형 데이터를 가장 잘 다루는 알고리즘이 바로 앙상블 학습이다.  
이것은 결정트리를 기반으로 만들어져있다.  
이번에 배울 것이 바로 이 앙상블 알고리즘이다.  

# 랜덤 포레스트
앙상블 학습의 대표 주자 중 하나로 가장 널리 사용된다.  
결정트리를 랜덤하게 만들어 결정 트리 숲을 형성한다.  
그리고 각 결정 트리의 예측을 통해 최종 예측을 구축한다.  
  
예를 들어, 1000개의 샘플 중 100개의 샘플을 뽑을 때, 하나를 먼저  
확인한 다음 다시 넣고 다시 샘플을 뽑는다면 중복이 발생할 수 있다.  
이렇게 만들어진 샘플을 부트스트랩 샘플이라고 한다.  
기본적으로, 부트스트랩 샘플은 훈련세트의 크기와 같게 한다.  
즉 1000개의 샘플이 있으면 1000번 샘플을 확인한다. (중복허용)  
  
이러한 랜덤 포레스트는 랜덤하게 선택한 샘플과 특성을 사용하기 때문에  
훈련세트에 과대적합되는 것을 막아주고 검증세트와 테스트 세트에서 안정적인  
성능을 얻을 수 있다.  


In [1]:
# 자료를 준비, 훈련세트와 테스트 세트로 분리
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol','sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size = 0.2, random_state = 42)

In [2]:
# cross_validate() 함수를 사용한 교차검증 수행
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs=-1, random_state = 42)
scores = cross_validate(rf, train_input, train_target, return_train_score= True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score'])) 

0.9973541965122431 0.8905151032797809


In [3]:
# 앞의 랜덤 포레스트 모델을 훈련시키고 특성 중요도를 출력
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


In [4]:
# 자체적으로 모델을 평가하는 점수
rf = RandomForestClassifier(oob_score = True, n_jobs = -1, random_state=42)
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


# 엑스트라 트리
엑스트라 트리는 랜덤 포레스트와 매우 비슷하기 동작한다.  
기본적으로 100개의 결정트리를 훈련한다.  
차이점은 부트스트랩 샘플을 사용하지 않는다는 점이다.  
즉, 결정 트리를 만들 때, 전체 훈련 세트를 사용한다.  
대신 노드를 분할할 때 가장 좋은 분할을 찾는 것이 아니라  
무작위분할을 한다.  
하나의 결정트리에서 특성을 무작위로 분할한다면 성능은 낮아지지만  
많은 트리를 앙상블하기 때문에 과대적합을 막고 검증세트 점수를  
높이는 효과가 있다.  
사이킷 런에서 제공하는 엑스트라 트리는 ExtraTreeClassifier다.  
이 모델의 교차검증 점수를 확인해보자

In [5]:
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs=-1, random_state =42)
scores = cross_validate(et, train_input, train_target, return_train_score = True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


여기서는 예제의 특성이 많지 않아 랜덤포레스트와 엑스트라 트리의 차이가 적다.  
보통 엑스트라 트리가 무작위성이 좀 더 크기 때문에 랜덤포레스트보다 더 많은  
결정트리를 훈련해야 한다.  
하지만 랜덤 노드 분할 덕분에 빠른 계산 속도가 장점이다.  

In [6]:
# 엑스트라 트리 훈련 후 특성 중요도 출력
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


# 그레이디언트 부스팅
깊이가 얕은 결정트리를 사용해서 이전트리의 오차를 보완하는 방식으로 하는 앙상블  
이른바 경사하강법을 사용해서 트리를 앙상블에 추가하는 방식이다.  
그레이언트 부스팅은 결정트리를 계속 추가하면서 가장 낮은 곳을 찾아 이동한다.  
때문에 깊이가 얕은 트리를 사용한다.

In [7]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state = 42)
scores = cross_validate(gb, train_input, train_target, return_train_score = True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [8]:
# 결정트리 개수를 500개로 늘림
gb = GradientBoostingClassifier(n_estimators= 500, learning_rate=0.2, random_state =42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [9]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15872278 0.68010884 0.16116839]


# 히스토그램 기반 그레이디언트 부스팅
히스토그램 기반 그레이디언트 부스팅은 정형데이터를 다루는 머신러닝 알고리즘 중  
가장 인기가 높은 알고리즘이다.  
먼저 입력틍성을 256개의 구간으로 나눈 다음 하나를 뗴어 놓고 누락된 값을 위해 사용한다.  
따라서 입력에 누락된 특성이 있더라도 이를 따로 전처리할 필요가 없다.  


In [10]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier(random_state = 42)
scores = cross_validate(hgb, train_input, train_target, return_train_score =True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

/usr/local/lib/python3.9/dist-packages/sklearn/experimental/enable_hist_gradient_boosting.py:16: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


0.9321723946453317 0.8801241948619236


In [11]:
from sklearn.inspection import permutation_importance
hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats = 10, random_state = 42, n_jobs = -1)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


In [12]:
result = permutation_importance(hgb, test_input, test_target, n_repeats = 10, random_state = 42, n_jobs = -1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [13]:
hgb.score(test_input, test_target)

0.8723076923076923

In [15]:
from xgboost import XGBClassifier
xgb = XGBClassifier(tree_method = 'hist', random_state = 42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9555033709953124 0.8799326275264677


# LightGBM²
널리 사용하는 또 다른 히스토그램 기반 그레이디언트 부스팅 라이브러리는  
마이크로소프트에서 개발한 LightGBM이다.  
이것은 빠르고 최신기술이 많이 적용되어 있어 인기가 점점 높아지고 있다.

In [17]:
from lightgbm import LGBMClassifier
lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score = True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.935828414851749 0.8801251203079884


# 정리
## ■ 앙상블 학습
앙상블 학습은 더 좋은 예측 결과를 만들기 위해 여러 개의 모델을 훈련하는   머신러닝 알고리즘이다.  
  
## ■ 랜덤 포레스트
대표적인 결정트리 기반의 앙상블 학습 방법, 부트스트랩 샘플을 사용하고  
랜덤하게 일부 특성을 선택해 트리를 만든다.  
  
## ■ 엑스트라 트리
랜덤 포레스트와 비슷하게 결정 트리를 사용해서 앙상블 모델을 만든다.  
그러나 부트스트랩 샘플을 사용하지 않는다.  
대신 랜덤 노드 분할을 통해 과대적합을 감소시킨다.  
  
## ■ 그레이디언트 부스팅
랜덤 포레스트나 엑스트라 트리와 달리 결정트리를 연속저으로 추가하여 손실 함수를  
최소화하는 앙상블 방법이다.  
때문에 훈련속도가 조금 느리지만 더 좋은 성능을 기대할 수 있다.  
그레이디언트 부스팅의 속도를 개선한 것이 히스토그램 기반 그레이디언트  부스팅이며 안정적인 결과와 높은 성능으로 인기가 높다.  


